# Preprocessing Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.image as img
import matplotlib.pyplot as plt
from tqdm import tqdm
import os, cv2

TRAIN_DIR = 'C:/Users/Brendon/URE18/images'
TEST_DIR = 'C:/Users/Brendon/URE18/Test_Images'
TRAIN_LABEL_DIR = 'Data_Entry_2017.csv'
TEST_LABEL_DIR = 'Data_Entry_2017.csv'
IMG_SIZE = 256

In [ ]:
# One Hot Encode for Multiple Labels
from sklearn.preprocessing import MultiLabelBinarizer

read_data = pd.read_csv(TRAIN_LABEL_DIR)
split_labels = [items.split('|')[0] for items in read_data.Finding_Labels]
one_hot = MultiLabelBinarizer()
training_labels = one_hot.fit_transform(split_labels)
training_labels = training_labels[101:1000]

read_data = pd.read_csv(TEST_LABEL_DIR)
split_labels = [items.split('|')[0] for items in read_data.Finding_Labels]
one_hot = MultiLabelBinarizer()
test_labels = one_hot.fit_transform(split_labels)
test_labels = test_labels[0:100]

In [2]:
def create_batch(directory):
    img_array = []
    for img in tqdm(os.listdir(directory)):
        path = os.path.join(directory, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        img_array.append(img)
    return img_array

In [5]:
training_img = create_batch(TRAIN_DIR)
training_img = np.array(training_img)
training_img = training_img[101:1000]

100%|██████████████████████████████████████| 4999/4999 [04:41<00:00, 17.76it/s]


In [4]:
test_img = create_batch(TEST_DIR)
test_img = np.array(test_img)
test_img = test_img[0:100]

100%|████████████████████████████████████████| 103/103 [00:01<00:00, 74.52it/s]


# Neural Network

In [6]:
import keras
from keras import models
from keras import layers

Using TensorFlow backend.


In [9]:
model = models.Sequential()
model.add(layers.Conv2D(32, (64, 64), activation='relu', input_shape=(256, 256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (12, 12), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (12, 12), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(13, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 193, 193, 32)      131104    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 85, 85, 64)        294976    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 42, 42, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 31, 31, 64)        589888    
_________________________________________________________________
flatten_2 (Flatten)          (None, 61504)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                3936320   
__________

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(training_img, training_labels, epochs = 1, batch_size = 100)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc